![](imgs/miredtwitter_header2.png)  
<center>
<h1>
<strike>Prediciendo</strike> Adivinando preferencias (individuales)
<br />y tendencias (masivas) en Twitter
</h1>
<h2>
(SNA, ML y algo de NLP)
</h2>
<h3>Pablo Gabriel Celayes</h3>
<h4>Miércoles 22 de agosto de 2018 - Seminario Machín Lenin</h4>
</center>

# Resumen

* Introducción

* Set the datos

* Predicción social

* Agregando NLP

* Popularidad


# Introducción: ¿quién soy?

* Lic. en Matemática (2006)

* Lic. en Ciencias de la Computación (2017)

* AI Engineer @Genesys ( desde 2017 )

* Dev / Data Scientist Freelance @Upwork ( desde 2013 )

![](imgs/miredtwitter.png)

# Introducción: ¿quiénes somos?

![](imgs/equipo.png)

# Introducción: cómo empezamos

## Idea original

- Recomendador *personalizado* de artículos basado en contenido ( NLP )

- Mejorarlo con información social ( de fuentes externas o relaciones *inferidas* ) 

- Datos explícitos sobre contenido visto y preferido por l@s usuari@s

- **Producto**

![](imgs/cogfor.png)

## Mutación

- Set the datos propio ( usuarios, preferencias, conexiones ) 

- Comenzar con predicción social de preferencias.

- Mejorar predicciones analizando el contenido.

- **Experimento**

# Datos: Grafo social

- hasta 3 pasos de la relación $\texttt{seguir}$ empezando de mí mismo

- filtramos usuarios **relevantes** ( >40 seguidos/seguidores )

- ~3M usuarios

- ~10M conexiones


# Datos: Contenido

- Subgrafo $G$:

    - Construcción:
    
        - partimos de un conjunto pequeño de usuarios
        
        - agregamos 50 *seguidos* más *afines* (según *seguidos* comunes)
        
        - repetimos hasta que no hay nuevos
    
    - Resultado:
        
        - 5180 usuarios
        
        - ~230k conexiones

# Datos: Contenido

- Tweets:

    - *timelines* entre 25/8/2015 y 24/9/2015 (+ *retweets* y *favs*)
    
    - 2M tweets ( 1,6M en castellano )

# Predicción social

¿ Dado un usuario $u$, cuánto puedo saber del **contenido** que **prefiere** sabiendo lo que prefieren los usuarios de su **entorno** social ?

- **Contenido** =  tweets en español _"visibles"_ ( $T_u$ )

- **Preferencias** = retweets

- **Entorno** = seguidos + seguidos-por-seguidos ( $E_u$ )


## Tweets visibles

* Compartidos por $u$ o sus seguidos

* Excluímos los _escritos_ por $u$

* máximo $10000$ ( submuestra de negativos si es necesario )

## Selección de usuari@s

* Entrenar y evaluar modelos es caro

* Soy vago y quería hacer todo en mi laptop

* Así que elegimos a quiénes analizar:

    * $A$ = $1000$ más *activos* (cant. de tweets)
    
    * $I$ = $1000$ más *centrales* (según Katz)
    
    * L@s afortunad@s: $A \cap I$ ($194$ usuari@s)


## Extracción de _features_

* $E_u = \{u_1, u_2, \ldots , u_n \}$ 

* $T_u = \{ t_1, \ldots, t_m \}$

$$
  M_u := [ \verb|tweet_en_tl|(t_i, u_j) ]_{\substack{ 1 \leq i \leq m \\ 1 \leq j \leq n}}  
$$


$$
  y_u := [ \texttt{tweet_en_tl}(t_i, u) ]_{ 1 \leq i \leq m }  
$$



## Problema de clasificación

* Predecir $y_u$ en base a filas de $M_u$

* Particionado:

    * $70\%$ entrenamiento ($M^{en}_u, y^{en}_u$)
    
    * $10\%$ ajuste ($M^{aj}_u, y^{aj}_u$)
    
    * $20\%$ evaluación ($M^{ev}_u, y^{ev}_u$)

## Support Vector Machines

* Objetivo: **maximizar** margen y **minimizar** errores

* Funciones *kernel* permiten encontrar fronteras **no lineales**:
    
    * *Radial Basis Function*
    
    * Polinómico

![](imgs/svm_linsep_err.png)



## Calidad de clasificación

$$\texttt{precision} := \frac{|\{x_i | f(x_i) = 1 \text{ y } y_i = 1 \}|}{|\{x_i | f(x_i) = 1 \}|}$$

$$ $$

$$\texttt{recall} := \frac{|\{x_i | f(x_i) = 1 \text{ y } y_i = 1 \}|}{|\{x_i | y_i = 1 \}|}$$

$$ $$

$$\texttt{F1} := \frac{2}{\frac{1}{\texttt{precision}} + \frac{1}{\texttt{recall}}} = \frac{2 * \texttt{precision} * \texttt{recall} }{\texttt{precision} + \texttt{recall}}$$



## Ajuste de hiper-parámetros

* Búsqueda exhaustiva $\verb|GridSearchCV|$

* Validación cruzada en $3$ partes

* Objetivo: maximizar $F1$

* Grilla:
```
{
    "C": [ 0.01, 0.1, 1 ],
    "class_weight": [ "balanced", None ], 
    "gamma": [ 0.1, 1, 10 ],
    "kernel": [ "rbf", "poly" ]
}
```
    * ```C```: controla balance entre margen y errores
    * ```class_weight```: ¿dar más importancia a clase minoritaria?
    * ```gamma```: forma de la frontera de decisión

## Resultados

* $F1$ promedio sobre $M^{ev}_u$ =  $87,7 \%$

![](imgs/f1s_social_valid.png)

# *¡Y sin tener en cuenta el contenido!*



![](imgs/robot2.png)

# Agregando NLP...

![](imgs/robotlee.jpg)

## Seguimos recortando...

* $F1 < 0,75$ en $M^{aj}_u$ ( $27$ usuarios )

* $10$ usuarios más (al azar)

## Pre-procesamiento

- Normalización

- Tokenizado

- Diccionario

- *Bag of words*

- LDA


## Normalización
- quitar URLs
- minúscula
- quitar acentos
- contraer repeticiones de vocales y espacios

![](imgs/normalize.png)

## Tokenizado

- partir en palabras

- remover puntuación

- _stemming_

- remover palabras de 1 caracter

![](imgs/tokenize.png)

## Diccionario

* Todo $T$ tokenizado. (**!!**)

* Significativas ( en al menos $100$ tweets ).

* Informativas ( en a lo sumo el $30\%$ de los tweets ).

* Diccionario $D$ de ~11K términos.



## *Bag of words*

- Texto $t$:  $\rightarrow$ multiconjunto (*bag*) de las palabras en $t$.

- No importa el orden, pero sí repeticiones.

- En tweets ( $\leq 140$ caracteres ), en general son *conjuntos* ( $0$ o $1$ ocurrencia).

- $D = \{ t_1, \ldots, t_{11000} \}$ : $\rightarrow$ vector de _features_ enteras (booleanas):

$$ v_{BOT}(tweet) = [count(t_i, tokens(tweet))]_{i=1}^{11000} $$

- $v_{BOT}(tweet)$ se representa ralo (*sparse*)


## LDA

* Descubre temas subyacentes en colecciones de textos
    
    * Tema: distribución de probabilidad de términos

* Permite asignar _scores_ por tema a cada documento ( tweet )

* Reducción de dimensionalidad ( espacio de *palabras* en $D$ a espacio de *temas* )


### Ejemplo: LDA 10 temas

![](imgs/lda10.png)

## Twitter-LDA

- Problema: tweets son cortos y normalmente hablan de un único tema 

- Adaptaciones:

    - Agrupar por usuario
    
    - Asignar un tema por tweet

## Evaluando sobre $M_u^{ev}$

![](imgs/f1s_social_vs_socialldas.png)


* Mejor modelo: $\texttt{TwitterLDA10}$ ( mejora media de $1,7\%$)

* Por ahora agregar NLP no mejora mucho

* Muchos casos mejoran en *train* pero decaen en *test* (*overfitting*)

# Predicción de popularidad

- Pasamos de

_dado un tuit t, un usuario u y el comportamiento de su entorno sobre t, predecir si u retuitea t_

a

_dado un tuit t, usuarios U y el comportamiento de influencers I sobre t, predecir si en U retuitean mucho t_

- Detección de influencers ( actividad + centralidad )

- word-embeddings con FastText

- mism@s usuari@s, más contenido


![](imgs/popularidad_f1s.png)

# ¿Cómo sigue?

- Considerar temporalidad, estudiar predicción temprana ( en progreso )

- Resolver overfitting en caso Social+NLP de predicción individual

- Generalizar ( modelo que no dependa de un usuario central fijo )

- ¡Más _features_ !

- ~~Instalar/voltear un gobierno vía Twitter~~ Profundizar el entendimiento de los procesos de formación de opinión y difusión de ideas en redes sociales

# ¡Gracias!


**Código** https://github.com/pablocelayes/sna_classifier/tree/micai_datos2015


**Tesis mía** https://www.dropbox.com/preview/Public/tesisSNA.pdf

**Tesis Matías Silva** https://drive.google.com/open?id=0BzBYYQ4vKWhgTlVvWUpJSmU3VEU4QkgtYWFWMG41czB5bkJN

**Paper** https://www.dropbox.com/preview/Public/retweet_prediction_micai_2017.pdf


**Twitter**: @PCelayes